In [0]:
%sql
DELETE FROM case_study_de.order_bronze;
WITH order_cte_v1 
(order_nk, order_timestamp, product_nk, customer_nk, revenue, order_status) 
AS (SELECT * FROM csv.`/Volumes/source_files/default/files/Order_1.csv`),
order_cte_v2 
(order_nk, order_timestamp, product_nk, customer_nk, revenue, order_status) 
AS (SELECT * FROM csv.`/Volumes/source_files/default/files/Order_2.csv`),
order_cte_v3 
(order_nk, order_timestamp, product_nk, customer_nk, revenue, order_status) 
AS (SELECT * FROM csv.`/Volumes/source_files/default/files/Order_3.csv`),
order_cte_v1_versioned AS 
(SELECT *, 1 as order_version FROM order_cte_v1),
order_cte_v2_versioned AS 
(SELECT *, 2 as order_version FROM order_cte_v2),
order_cte_v3_versioned AS 
(SELECT *, 3 as order_version FROM order_cte_v3),
order_union AS (
  SELECT * FROM order_cte_v1_versioned 
  UNION ALL 
  SELECT * FROM order_cte_v2_versioned
  UNION ALL 
  SELECT * FROM order_cte_v3_versioned
  ),
order_transformed AS (SELECT 
  order_nk,
  try_to_timestamp(substring(order_timestamp, 1, 26), 'yyyy/MM/dd HH:mm:ss.SSSSSS') 
  AS order_timestamp,
  product_nk,
  customer_nk,
  TRY_CAST(REPLACE(customer_nk, 'CUST', '') AS INT) AS customer_nk_num,
  order_version AS customer_version,
  TRY_CAST(revenue AS DECIMAL(10,2)) AS revenue,
  order_status,
  TRY_CAST(REPLACE(order_nk, 'ORD', '') AS INT) AS order_nk_num,
  order_version,
  TRY_CAST(REPLACE(product_nk, 'PRD', '') AS INT) AS product_nk_num,
  order_version AS product_version
FROM order_union),
order_filtered AS (SELECT * FROM order_transformed WHERE order_nk_num IS NOT NULL)
INSERT INTO case_study_de.order_bronze (
  order_nk,
  order_timestamp,
  product_nk,
  customer_nk,
  customer_nk_num,
  customer_version,
  revenue,
  order_status,
  order_nk_num,
  order_version,
  product_nk_num,
  product_version
)
SELECT 
  order_nk,
  order_timestamp,
  product_nk,
  customer_nk,
  customer_nk_num,
  customer_version,
  revenue,
  order_status,
  order_nk_num,
  order_version,
  product_nk_num,
  product_version   
FROM order_filtered;


